In [10]:
import pandas as pd

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(1)

In [11]:
# read data
df = pd.read_csv('data/Strategic_Subject_List_-_Historical.csv')
df.head()

/tmp/ipykernel_231/152499078.py:2: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Strategic_Subject_List_-_Historical.csv')


,SSL SCORE,PREDICTOR RAT AGE AT LATEST ARREST,PREDICTOR RAT VICTIM SHOOTING INCIDENTS,PREDICTOR RAT VICTIM BATTERY OR ASSAULT,PREDICTOR RAT ARRESTS VIOLENT OFFENSES,PREDICTOR RAT GANG AFFILIATION,PREDICTOR RAT NARCOTIC ARRESTS,PREDICTOR RAT TREND IN CRIMINAL ACTIVITY,PREDICTOR RAT UUW ARRESTS,SEX CODE CD,...,RAW_HEAT_SCORE,STATUS I,PRE RAW HEAT SCORE,TRAP FLAGS,SSL FLAGS,LATITUDE,LONGITUDE,CENSUS TRACT,COMMUNITY AREA,LOCATION
0,306,20-30,0,0,0,0,0,-0.3,0,M,...,0.09110,Y,9110,NaN,NaN,NaN,NaN,NaN,,NaN
1,352,20-30,0,1,0,0,0,0.0,0,M,...,0.09949,Y,9949,NaN,NaN,NaN,NaN,NaN,,NaN
2,349,20-30,0,0,2,0,2,0.0,0,M,...,0.09908,Y,9908,NaN,NaN,NaN,NaN,NaN,,NaN
3,344,20-30,0,0,3,0,0,-0.3,0,M,...,0.09810,Y,9810,NaN,NaN,NaN,NaN,NaN,,NaN
4,343,20-30,0,0,1,0,0,0.4,0,M,...,0.09794,Y,9794,NaN,NaN,NaN,NaN,NaN,,NaN


In [12]:
# drop unnecessary columns
df.drop(['STATUS I', 'MAJORITY DIST', 'DLST', 'IDOC RES CITY', 'IDOC RES STATE CODE', 'IDOC RES ZIP CODE', 'IDOC CPD DIST', 'TRAP FLAGS', 'SSL FLAGS'], axis=1, inplace = True) #IDOC RES CITY (typos)
df.drop(['HEAT SCORE', 'RAW_HEAT_SCORE', 'RAW SSL SCORE', 'PRE RAW HEAT SCORE'], axis=1, inplace = True) # same as SSL score
df.drop(['PREDICTOR RAT AGE AT LATEST ARREST', 'PREDICTOR RAT VICTIM SHOOTING INCIDENTS', 'PREDICTOR RAT VICTIM BATTERY OR ASSAULT', 'PREDICTOR RAT ARRESTS VIOLENT OFFENSES', 'PREDICTOR RAT GANG AFFILIATION', 'PREDICTOR RAT NARCOTIC ARRESTS', 'PREDICTOR RAT TREND IN CRIMINAL ACTIVITY', 'PREDICTOR RAT UUW ARRESTS'], axis=1, inplace = True) # explicit columns used to calculate SSL
df.drop(['TRAP STATUS', 'LATITUDE', 'LONGITUDE', 'LOCATION'], axis=1, inplace = True) # might still want to include

In [13]:
# fill and/or remove na values
df["STOP ORDER NO"] = np.where(df["STOP ORDER NO"]>0 , 1, df["STOP ORDER NO"]) # 0 if false, 1 if true
df["STOP ORDER NO"].fillna("0", inplace = True)

df["SSL LAST PTV DATE"].fillna("0", inplace = True)
df["LATEST DOMESTIC ARR DATE"].fillna("0", inplace = True)
df["LATEST NARCOTIC ARR DATE"].fillna("0", inplace = True)
df["LATEST WEAPON ARR DATE"].fillna("0", inplace = True)
df["PAROLEE I"].fillna("N", inplace = True)
df["WEAPONS ARR CNT"].fillna("0", inplace = True)
df["NARCOTICS ARR CNT"].fillna("0", inplace = True)
df["DOMESTIC ARR CNT"].fillna("0", inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398684 entries, 0 to 398683
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SSL SCORE                 398684 non-null  int64  
 1   SEX CODE CD               398684 non-null  object 
 2   RACE CODE CD              398684 non-null  object 
 3   WEAPON I                  398684 non-null  object 
 4   DRUG I                    398684 non-null  object 
 5   AGE GROUP                 398531 non-null  object 
 6   AGE TO                    398531 non-null  object 
 7   STOP ORDER NO             398684 non-null  object 
 8   PAROLEE I                 398684 non-null  object 
 9   LATEST DATE               398684 non-null  int64  
 10  LATEST DIST               398684 non-null  int64  
 11  LATEST DIST RES           398684 non-null  int64  
 12  WEAPONS ARR CNT           398684 non-null  object 
 13  LATEST WEAPON ARR DATE    398684 non-null  o

In [14]:
# scaling of column values
df['SSL SCORE'] = np.where(df['SSL SCORE']<250 , 0, df['SSL SCORE']) # high risk = 1, low risk = 0
df['SSL SCORE'] = np.where(df['SSL SCORE']>=250 , 1, df['SSL SCORE'])

df['RACE CODE CD'] = np.where(df['RACE CODE CD'] == 'WHI' , 1, df['RACE CODE CD']) # white = 1, non-white = 0
df['RACE CODE CD'] = np.where(df['RACE CODE CD'] == 'BLK', 0, df['RACE CODE CD'])
df['RACE CODE CD'] = np.where(df['RACE CODE CD'] == 'WWH', 0, df['RACE CODE CD'])
df['RACE CODE CD'] = np.where(df['RACE CODE CD'] == 'API', 0, df['RACE CODE CD'])
df['RACE CODE CD'] = np.where(df['RACE CODE CD'] == 'WBH', 0, df['RACE CODE CD'])
df['RACE CODE CD'] = np.where(df['RACE CODE CD'] == 'I', 0, df['RACE CODE CD'])
df = df[df['RACE CODE CD'] != 'U']

df['SEX CODE CD'] = np.where(df['SEX CODE CD'] == 'M', 1, df['SEX CODE CD']) # male = 1, female = 0
df['SEX CODE CD'] = np.where(df['SEX CODE CD'] == 'F', 0, df['SEX CODE CD'])
df = df[df['SEX CODE CD'] != 'X']

'20-30', '50-60', '30-40', 'less than 20', '60-70', '40-50', '70-80'

df['AGE GROUP'] = np.where(df['AGE GROUP'] == 'less than 20' , 0, df['AGE GROUP']) # order of ages, '20-30', '50-60', '30-40', 'less than 20', '60-70', '40-50', '70-80'
df['AGE GROUP'] = np.where(df['AGE GROUP'] == '20-30', 1, df['AGE GROUP'])
df['AGE GROUP'] = np.where(df['AGE GROUP'] == '30-40', 2, df['AGE GROUP'])
df['AGE GROUP'] = np.where(df['AGE GROUP'] == '40-50', 3, df['AGE GROUP'])
df['AGE GROUP'] = np.where(df['AGE GROUP'] == '50-60', 4, df['AGE GROUP'])
df['AGE GROUP'] = np.where(df['AGE GROUP'] == '60-70', 5, df['AGE GROUP'])
df['AGE GROUP'] = np.where(df['AGE GROUP'] == '70-80', 6, df['AGE GROUP'])

# look into 'SSL LAST PTV DATE', 

/tmp/ipykernel_231/3633432863.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SEX CODE CD'] = np.where(df['SEX CODE CD'] == 'M', 1, df['SEX CODE CD']) # male = 1, female = 0
/tmp/ipykernel_231/3633432863.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SEX CODE CD'] = np.where(df['SEX CODE CD'] == 'F', 0, df['SEX CODE CD'])
/tmp/ipykernel_231/3633432863.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [15]:
# drop columns with null values

# 'CENSUS TRACT'
df.dropna(inplace=True)

/tmp/ipykernel_231/4126566958.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [16]:
df['LATEST DOMESTIC ARR DATE'].unique()

array(['0', 2014.0, 2010.0, 2013.0, 2012.0, 2011.0, 2016.0, 2006.0,
       2015.0, 2008.0, 2007.0, 2009.0], dtype=object)

In [17]:
df['CENSUS TRACT'].unique()

array([0.00000000e+00, 1.61100000e+05, 8.39100000e+05, 9.80000000e+05,
       1.70310626e+10, 1.70310630e+10, 1.70310621e+10, 1.70310633e+10,
       1.70310703e+10, 1.70310702e+10, 1.70310701e+10, 1.70310629e+10,
       1.70310707e+10, 1.70310634e+10, 1.70310704e+10, 1.70310701e+10,
       1.70310101e+10, 1.70312519e+10, 1.70310104e+10, 1.70310202e+10,
       1.70310204e+10, 1.70310303e+10, 1.70310206e+10, 1.70310207e+10,
       1.70310205e+10, 1.70310206e+10, 1.70310203e+10, 1.70310208e+10,
       1.70310207e+10, 1.70310102e+10, 1.70310106e+10, 1.70310103e+10,
       1.70310105e+10, 1.70310105e+10, 1.70310105e+10, 1.70310102e+10,
       1.70312604e+10, 1.70310302e+10, 1.70310209e+10, 1.70310301e+10,
       1.70310208e+10, 1.70310305e+10, 1.70310306e+10, 1.70310306e+10,
       1.70310306e+10, 1.70310301e+10, 1.70310107e+10, 1.70310107e+10,
       1.70310203e+10, 1.70310201e+10, 1.70310312e+10, 1.70310307e+10,
       1.70310314e+10, 1.70310315e+10, 1.70310313e+10, 1.70310315e+10,
      

In [18]:
df.to_csv('data/preprocessed_data.csv', index=False)